In [1]:
# get ic decay
from get_data import get_data,bars
from singletrader.processors.cs_processor import CsWinzorize
from singletrader.shared.utility import save_pkl
from singletrader.factorlib import FactorEvaluation,summary_plot
import pandas as pd
import plotly.express as px
from plotly.figure_factory import create_table
import warnings
from tools.workflow import MultiFactorTesting
warnings.filterwarnings('ignore')





{"level": "INFO", "time": "2023-03-27 16:31:08,942", "timeMillis": 1679905868839,"module": "numexpr.utils", "method": "python-service", "addition": "[utils.py 148 _init_num_threads]", "message": "Note: NumExpr detected 32 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8."}
{"level": "INFO", "time": "2023-03-27 16:31:08,943", "timeMillis": 1679905868839,"module": "numexpr.utils", "method": "python-service", "addition": "[utils.py 160 _init_num_threads]", "message": "NumExpr defaulting to 8 threads."}
{"level": "INFO", "time": "2023-03-27 16:31:09,117", "timeMillis": 1679905868839,"module": "singletrader-service", "method": "python-service", "addition": "[__init__.py 26 init]", "message": "开始初始化路径..."}
[3404:MainThread](2023-03-27 16:31:09,439) INFO - qlib.Initialization - [config.py:416] - default_conf: client.
{"level": "INFO", "time": "2023-03-27 16:31:09,439", "timeMillis": 1679905868839,"module": "qlib.Initialization", "method": "python-service", "addition": "[c

In [2]:
data = get_data().dropna() #获取处理好的数据
features = [_f for _f in data.columns if _f not in bars] + ['ep','circulating_market_cap']
bar_data = data[bars]
# bar_data.head()

# Tips
-   默认的极值处理方式均调整为为0.01/0.99放缩，前值为5倍标准差处理

In [3]:
liquidity_group = data.groupby(level=0).apply(lambda x:pd.qcut(x['amount3M'],3,labels=['low','medium','high'])).droplevel(0)
ep_group = data.groupby(level=0).apply(lambda x:pd.qcut(x['ep'],3,labels=['low','medium','high'])).droplevel(0)
high_liq =(liquidity_group=='high').astype(int)
print('high liquidity set\n',high_liq.head())

high liquidity set
 date        code       
2006-01-31  000001.XSHE    1
            000002.XSHE    1
            000004.XSHE    0
            000005.XSHE    0
            000006.XSHE    0
Name: amount3M, dtype: int32


# summary of all factors

- 非行业中性化(全市场)

In [5]:
# 非行业中性化(全市场)0
fe = FactorEvaluation(bar_data=data[bars],factor_data=data[features],freq=12,winzorize={'method':'qtile','k':0.01},standardize=True,industry_neutralize=False)
fe.factor_data['comp_mom12x3_to3M'] = fe.factor_data['mom12x3'] - fe.factor_data['turnover3M']
fe.get_summary(add_shift=0,start_date='2005-01-01',end_date='2022-12-31',base='close',groups=10) # forward_return: 下月收盘-本月收盘
# fe.get_summary(add_shift=0,start_date='2010-01-01',end_date='2022-12-31',universe=high_liq,base='close-open') # forward_return: 下月收盘-下月开盘

start_date:2005-01-01 / end_date:2022-12-31


,skew,adjskew,distance,mom6x3,mom9x3,mom12x3,mom1M,mom2M,mom3M,mom6M,turnover3M,amount3M,stddev_diff,stddev,ep,circulating_market_cap,comp_mom12x3_to3M
ic.mean,-0.0208,-0.0232,0.0043,-0.0046,-0.0017,-0.0000,-0.0469,-0.0503,-0.0457,-0.0384,-0.0430,-0.0421,-0.0069,-0.0397,0.0100,-0.0182,0.0327
ic.t-stats,-4.6902,-5.5927,0.4000,-0.6455,-0.2177,-0.0039,-5.7243,-5.3496,-4.7678,-3.9573,-4.3850,-5.2076,-1.3862,-3.5242,1.4971,-2.1585,3.7827
AnnRet_Long,-0.0563,-0.0635,0.0194,-0.0484,-0.0347,-0.0063,-0.1594,-0.1587,-0.1545,-0.1407,-0.1393,-0.1373,-0.0624,-0.1483,0.0282,-0.0949,0.0380
AnnRet_short,0.0092,0.0284,0.0338,-0.0049,-0.0412,-0.0228,0.0362,0.0916,0.0986,0.0874,0.1018,0.1787,-0.0490,0.0845,0.0152,0.1880,-0.1401
AnnRet,-0.0655,-0.0919,-0.0144,-0.0435,0.0066,0.0165,-0.1956,-0.2503,-0.2531,-0.2280,-0.2411,-0.3161,-0.0134,-0.2328,0.0130,-0.2829,0.1781
SR_Long,-0.9702,-1.2841,0.1556,-0.4726,-0.3128,-0.0523,-1.4038,-1.2294,-1.1370,-1.0549,-1.2448,-1.0432,-0.8368,-1.3053,0.2000,-0.6079,0.2935
SR_Short,0.1681,0.5336,0.2204,-0.0508,-0.4735,-0.2415,0.3551,0.7746,0.7834,0.6943,0.5594,1.1502,-0.7271,0.4533,0.1488,1.5089,-1.3748
SR,-0.6969,-1.0640,-0.0571,-0.2580,0.0378,0.0874,-1.0598,-1.1517,-1.1040,-0.9725,-0.9043,-1.3596,-0.1174,-0.8887,0.0713,-1.1362,0.8916
TO_Long,0.8818,0.8864,0.3014,0.5186,0.3729,0.3168,0.8677,0.6283,0.5174,0.3719,0.2486,0.1672,0.9529,0.7038,0.1153,0.0560,0.3129
TO_Short,0.8804,0.8819,0.5052,0.5962,0.4513,0.3740,0.8966,0.7005,0.5968,0.4524,0.1830,0.2377,0.9505,0.4906,0.0880,0.1121,0.3266


- 行业中性化(全市场)

In [12]:
# 行业中性化(全市场)
fe = FactorEvaluation(bar_data=data[bars],factor_data=data[features],freq=12,winzorize={'method':'qtile','k':0.01},standardize=True,industry_neutralize=True)
fe.factor_data['comb_mom12x3_to3M_GN'] = fe.factor_data['mom12x3_GN'] - fe.factor_data['turnover3M_GN']
fe.get_summary(add_shift=0,start_date='2006-01-01',end_date='2022-12-31',base='close',groups=10)[['mom12x3_GN','turnover3M_GN','comb_mom12x3_to3M_GN']]

start_date:2006-01-01 / end_date:2022-12-31


KeyError: "['turnover12M_GN', 'comb_mom12x3_to3M_GN'] not in index"

-   行业中性化因子（高流动性股票池）

In [7]:
# 行业中性化因子（高流动性股票池）
fe = FactorEvaluation(bar_data=data[bars],factor_data=data[features],freq=12,winzorize={'method':'qtile','k':0.01},standardize=True,industry_neutralize=True)
fe.factor_data['comp_mom12x3_to3M'] = fe.factor_data['mom12x3_GN'] - fe.factor_data['turnover3M_GN']
fe.get_summary(add_shift=0,start_date='2010-01-01',universe=high_liq,end_date='2022-12-31',base='close',groups=10) # forward_return: 下月收盘-本月收盘

start_date:2010-01-01 / end_date:2022-12-31


,skew_GN,adjskew_GN,distance_GN,mom6x3_GN,mom9x3_GN,mom12x3_GN,mom1M_GN,mom2M_GN,mom3M_GN,mom6M_GN,turnover3M_GN,amount3M_GN,stddev_diff_GN,stddev_GN,ep_GN,circulating_market_cap_GN,comp_mom12x3_to3M
ic.mean,-0.0204,-0.0137,-0.0039,0.0173,0.0207,0.0230,-0.0471,-0.0486,-0.0395,-0.0173,-0.0490,-0.0210,0.0041,-0.0301,0.0340,0.0154,0.0540
ic.t-stats,-4.1446,-2.8240,-0.3410,2.0095,2.1907,2.3551,-5.0088,-4.6888,-3.9642,-1.6545,-4.0443,-2.7782,0.6717,-2.5441,3.5670,1.3446,5.1568
AnnRet_Long,-0.0642,-0.0377,0.0033,0.0294,0.0444,0.0434,-0.1322,-0.1203,-0.1065,-0.0714,-0.1418,-0.0493,-0.0438,-0.1241,0.0509,0.0335,0.0853
AnnRet_short,0.0336,0.0307,0.0008,-0.0482,-0.0740,-0.0639,0.0471,0.0903,0.0703,0.0060,0.0654,0.0221,-0.0644,0.0150,-0.0950,-0.0375,-0.1452
AnnRet,-0.0978,-0.0684,0.0025,0.0775,0.1184,0.1073,-0.1793,-0.2106,-0.1768,-0.0774,-0.2072,-0.0714,0.0206,-0.1392,0.1459,0.0710,0.2305
SR_Long,-1.0015,-0.7492,0.0279,0.2682,0.3843,0.3710,-1.1896,-1.0204,-0.9405,-0.5758,-1.0922,-0.4345,-0.5759,-1.0548,0.3434,0.2082,0.7441
SR_Short,0.5624,0.5526,0.0074,-0.6271,-0.8787,-0.8289,0.5146,0.9460,0.6993,0.0655,0.6751,0.3645,-0.8217,0.1499,-1.1369,-0.5263,-1.2063
SR,-1.0853,-0.7821,0.0127,0.5329,0.7427,0.6591,-1.0559,-1.1925,-1.0203,-0.4271,-1.0169,-0.5382,0.1716,-0.7099,0.8285,0.3737,1.2183
TO_Long,0.9034,0.9068,0.3885,0.5595,0.4276,0.3739,0.8908,0.7207,0.5926,0.4336,0.3300,0.1962,0.9742,0.7320,0.1963,0.0685,0.3691
TO_Short,0.9059,0.9087,0.5766,0.6421,0.5307,0.4710,0.9164,0.7609,0.6404,0.4894,0.2510,0.4390,0.9661,0.6433,0.2657,0.2669,0.4310


# double sort (mom12x3 - turnover3M) universe = high liquidity

-   row data(3x3)

In [8]:
fe = FactorEvaluation(bar_data=data[bars],factor_data=data[features],freq=12,winzorize={'method':'qtile','k':0.01},standardize=True,industry_neutralize=False)
mft = MultiFactorTesting(bar_data=data.dropna()[bars],market_cap_col='circulating_market_cap',universe=high_liq,factor_data=fe.factor_data[['mom12x3','turnover3M']],add_shift=0)
summary = mft.summary(quantiles=3)
for i in list(summary.keys())[:2]+['avg_number_of_stocks']:print(i,'\n',summary[i])

avg_annual_excess_return_eq(%)_with_mkt=3.82% 
 turnover3M     1     2      3
mom12x3                      
1          -2.23 -3.48 -10.41
2           4.45  2.21  -5.51
3           8.90  7.08  -1.30
avg_anual_excess_return_cap(%)_with_mkt=3.44% 
 turnover3M     1     2      3
mom12x3                      
1          -3.61 -5.93 -13.23
2           4.97  0.59  -8.93
3           3.48  3.61  -3.97
avg_number_of_stocks 
 turnover3M       1       2       3
mom12x3                           
1           118.65  105.60   95.44
2           113.63  106.21   99.19
3            87.41  107.21  124.69


-   raw data(5x5) 

In [9]:
summary = mft.summary(quantiles=5)
for i in list(summary.keys())[:2]+['avg_number_of_stocks']:print(i,'\n',summary[i])

avg_annual_excess_return_eq(%)_with_mkt=3.82% 
 turnover3M      1      2      3     4      5
mom12x3                                     
1           -4.96  -4.31  -5.19 -7.83 -17.02
2            1.73   0.93   1.05 -2.19  -8.25
3            6.20   1.24  -0.19 -1.01  -8.84
4            5.19   6.55   1.74  0.45  -5.37
5           12.28  12.73  10.25  5.86  -4.11
avg_anual_excess_return_cap(%)_with_mkt=3.44% 
 turnover3M     1      2     3     4          5
mom12x3                                       
1          -5.92  -6.53 -7.75 -9.68 -18.879999
2           0.15  -0.88 -0.01 -4.13 -12.730000
3           7.06  -3.29 -2.99 -5.42 -11.960000
4           2.87   4.09 -2.26 -3.29  -9.000000
5           5.37  11.20  5.55  1.66  -8.760000
avg_number_of_stocks 
 turnover3M      1      2      3      4      5
mom12x3                                      
1           41.01  40.63  38.82  36.54  34.99
2           47.77  39.93  37.61  34.96  31.10
3           42.30  38.57  38.32  36.70  35.56
4      

-   industry nutrualized data(3x3)


In [10]:
fe = FactorEvaluation(bar_data=data[bars],factor_data=data[features],freq=12,winzorize={'method':'qtile','k':0.01},standardize=True,industry_neutralize=True)
mft = MultiFactorTesting(bar_data=data.dropna()[bars],market_cap_col='circulating_market_cap',universe=high_liq,factor_data=fe.factor_data[['mom12x3_GN','turnover3M_GN']],add_shift=0)
summary = mft.summary(quantiles=3)
for i in list(summary.keys())[:2]+['avg_number_of_stocks']:print(i,'\n',summary[i])

avg_annual_excess_return_eq(%)_with_mkt=3.82% 
 turnover3M_GN     1     2     3
mom12x3_GN                     
1             -2.02 -1.16 -9.40
2              4.90  2.44 -6.22
3              8.97  5.49 -1.52
avg_anual_excess_return_cap(%)_with_mkt=3.44% 
 turnover3M_GN     1     2      3
mom12x3_GN                      
1             -3.02 -3.05 -12.47
2              4.10  2.33 -10.12
3              5.22  1.86  -3.82
avg_number_of_stocks 
 turnover3M_GN       1       2       3
mom12x3_GN                           
1              113.11  102.60   95.60
2              109.47  108.87   92.28
3               88.73   99.15  123.02


-  industry nutrualized data(5x5)


In [11]:
summary = mft.summary(quantiles=5)
for i in list(summary.keys())[:2]+['avg_number_of_stocks']:print(i,'\n',summary[i])

avg_annual_excess_return_eq(%)_with_mkt=3.82% 
 turnover3M_GN      1     2     3     4      5
mom12x3_GN                                   
1              -1.74 -1.96 -2.98 -5.05 -11.48
2               2.02 -2.34  0.45 -2.87 -10.43
3               7.64  3.59  2.46 -3.03  -8.62
4               4.62  6.95  3.69 -1.38  -4.45
5              14.53  9.65  5.32  6.83  -5.17
avg_anual_excess_return_cap(%)_with_mkt=3.44% 
 turnover3M_GN     1     2     3     4      5
mom12x3_GN                                  
1             -1.87 -3.33 -4.61 -6.34 -15.61
2             -0.31 -5.02 -0.44 -7.11 -16.77
3              5.91  1.22  0.38 -8.10  -9.42
4              2.16  8.88  3.39 -6.53  -8.33
5              7.90  6.50  2.81  3.47  -8.48
avg_number_of_stocks 
 turnover3M_GN      1      2      3      4      5
mom12x3_GN                                      
1              40.69  37.04  35.82  37.23  36.21
2              42.83  41.31  38.59  33.50  30.13
3              38.70  41.00  39.81  34.83  32.00

# details of mom12x3_GN
收益曲线和表格指标，均是针对超额收益，benchmark用的是equal weighted

In [17]:
fe.all_factors

['skew_GN',
 'adjskew_GN',
 'distance_GN',
 'mom6x3_GN',
 'mom9x3_GN',
 'mom12x3_GN',
 'mom1M_GN',
 'mom2M_GN',
 'mom3M_GN',
 'mom6M_GN',
 'turnover3M_GN',
 'amount3M_GN',
 'stddev_diff_GN',
 'stddev_GN',
 'ep_GN',
 'circulating_market_cap_GN',
 'comp_mom12x3_to3M']

In [9]:
factor='comp_mom12x3_to3M'
report = fe.get_factor_detail_report(factor=factor,universe=high_liq,add_shift=0,base='close',start_date='2006-01-01',end_date='2022-12-31',total=False,excess_return=True,holding_period=1)
summary_plot(report)


In [10]:
group_ic = fe.factor_ana(factor=factor,ep_group=ep_group,liquidity_group=liquidity_group,add_shift=0,base='close')
bar = px.bar(group_ic['ic.mean'].reset_index(),x='level_1',y='ic.mean',color='level_0',barmode='group',title='ic of different ep & liquidity')
bar.show()

